<a href="https://colab.research.google.com/github/Dayaanaly/Dayaanaly/blob/main/L10_Transferencia_de_Aprendizaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transferencia de aprendizaje
## Dr. Carlos Villaseñor

Importar bibliotecas



In [17]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importar MobileNet base pero descartaremos las últimas capa

In [19]:
base_model=MobileNet(weights='imagenet',include_top=False)

<ipython-input-19-38e91c8c2631>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model=MobileNet(weights='imagenet',include_top=False)


Crear nueva Red Neuronal

In [20]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(256,activation='relu')(x)
x=Dense(64,activation='relu')(x)
preds=Dense(5, activation='softmax')(x)

Pegar Redes Neuronales

In [21]:
model=Model(inputs=base_model.input, outputs=preds)

Definir que capas se entrenan

In [22]:
for layer in model.layers[:-5]:
    layer.trainable=False
for layer in model.layers[-5:]:
    layer.trainable=True

Cargar datos nuevos

- Carge el archivo 'train.zip' en la pestaña archivos en el lateral izquierdo
- Corra la siguiente linea para descomprimir los archivos

In [31]:
!wget "https://drive.google.com/drive/u/2/folders/1YlwoyENoga0aV38I7f_wsTnWj32oFMhS"

--2025-04-27 08:37:32--  https://drive.google.com/drive/u/2/folders/1YlwoyENoga0aV38I7f_wsTnWj32oFMhS
Resolving drive.google.com (drive.google.com)... 74.125.137.102, 74.125.137.101, 74.125.137.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.137.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/drive/folders/1YlwoyENoga0aV38I7f_wsTnWj32oFMhS [following]
--2025-04-27 08:37:33--  https://drive.google.com/drive/folders/1YlwoyENoga0aV38I7f_wsTnWj32oFMhS
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&osid=1&continue=https://drive.google.com/drive/folders/1YlwoyENoga0aV38I7f_wsTnWj32oFMhS&followup=https://drive.google.com/drive/folders/1YlwoyENoga0aV38I7f_wsTnWj32oFMhS [following]
--2025-04-27 08:37:33--  https://accounts.google.com/ServiceLogin?service=wise&passive=1209600&osid=1&co

In [38]:
!unzip train.zip -d'/content/drive/MyDrive/Tarea IA/'

Archive:  train.zip
   creating: /content/drive/MyDrive/Tarea IA/train/
   creating: /content/drive/MyDrive/Tarea IA/train/cats/
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/2Q== (1).jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/2Q== (2).jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/2Q== (3).jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/2Q== (4).jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/2Q==.jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/9k= (1).jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/9k= (2).jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/9k=.jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/images (1).jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/images (10).jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/images (100).jpg  
  inflating: /content/drive/MyDrive/Tarea IA/train/cats/images (11).jpg  
  inflating: /cont

In [41]:
# Entrenamiento con augmentaciones
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2    # <-- ojo aquí
)

train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Tarea IA/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',      # 80% para entrenar
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Tarea IA2/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',    # 20% para validar
    shuffle=False
)

Found 158 images belonging to 3 classes.
Found 0 images belonging to 0 classes.


In [42]:
# Entrenamiento con augmentaciones
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2    # <-- ojo aquí
)

train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Tarea IA/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',      # 80% para entrenar
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Tarea IA2/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',    # 20% para validar
    shuffle=False
)

Found 158 images belonging to 3 classes.
Found 0 images belonging to 0 classes.


Crear generador desde carpeta

In [43]:
# Entrenamiento con augmentaciones
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2    # <-- ojo aquí
)

train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Tarea IA/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',      # 80% para entrenar
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Tarea IA2/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',    # 20% para validar
    shuffle=False
)

Found 158 images belonging to 3 classes.
Found 0 images belonging to 0 classes.


Compilar Modelo y entrenar desde carpeta

In [44]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',
              metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size

model.fit(train_generator,
          steps_per_epoch=step_size_train,
          epochs=6)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/6


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 3), output.shape=(None, 5)